In [11]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
from  TCGA_files import queryFile
from matplotlib import pyplot as plt

In [12]:
working_dir = "/Users/filippo/Developer/tesi"
os.chdir(working_dir)

In [13]:
df = pd.read_csv(("%s/genes.txt"%working_dir), header=[0], index_col=[0])
print(len(df))
df.head()

60483


,lenght,type_of_gene
ENSG00000242268,NaN,intergenic
ENSG00000270112,NaN,unknown
ENSG00000167578,18722.0,protein-coding
ENSG00000273842,NaN,unknown
ENSG00000078237,46830.0,protein-coding


In [14]:
uniq = [g[:15] for g in df.index.values]

In [15]:
len(np.unique(uniq))

60483

In [16]:
#q_many = mg.getgenes(df[df['type_of_gene']=='protein-coding'].index.values,'symbol,genomic_pos,genomic_pos_hg19')
#df_lenghts = df[df['type_of_gene']=='protein-coding']
#df_lenghts = df
#df_lenghts.insert(0,'lenght',lenghts)
#df_lenghts.to_csv("genes_l.txt",header=True,index=True)

In [17]:
q_many = pd.read_csv("genes.txt", index_col=[0], header=[0])
q_many = q_many[q_many['type_of_gene']=='protein-coding']
lenghts = q_many['lenght']

In [18]:
df = pd.read_csv(("%s/mainTable.csv"%working_dir), index_col=[0])

In [19]:
df.head()

,d2587070-cb7d-440d-ae49-52f5077248e6.htseq.counts.gz,97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz,646d58e7-b7e2-4b2b-bb37-3ca20b601213.htseq.counts.gz,0f2b58b4-7d91-48f3-8901-b624e3d6aa51.htseq.counts.gz,3eb03db7-efc1-4f20-bbc9-61551905eb9b.htseq.counts.gz,73ce143e-44f1-49c6-963f-45246d8551de.htseq.counts.gz,09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz,6ae2dd6c-2a39-411f-a1fc-11e0e6e82165.htseq.counts.gz,5f5c42f3-3631-43a3-9774-164eca819389.htseq.counts.gz,b520edf2-42ce-4b64-8575-19e34b30f042.htseq.counts.gz,...,37e498a7-b147-4c38-a3f5-ed283d57f97a.htseq.counts.gz,c264fe3b-482b-44ec-83a4-73df565663ff.htseq.counts.gz,9e97f017-6855-4d68-bd2a-c4da3ccb1d57.htseq.counts.gz,15864159-be88-41c8-bdef-c2c5927cb1a1.htseq.counts.gz,e2bdca76-7bc5-4143-a6f6-3c6642425c65.htseq.counts.gz,b1868d9d-ef06-4353-883b-230dae23f8b6.htseq.counts.gz,2b5f6764-5a5b-47dc-b0f8-7a548bab2e26.htseq.counts.gz,7324b2d6-dd30-4f99-a101-7c3d497bbcb8.htseq.counts.gz,95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz,00cc9b4d-a847-464e-979a-7751e1a87ae3.htseq.counts.gz
ENSG00000276289,0,0,0,0,4,0,1,0,0,0,...,0,0,0,25,2,0,1,1,5,1
ENSG00000137948,1,0,0,0,1,0,0,1,2,3,...,2,0,0,0,0,0,1,2,0,0
ENSG00000057149,0,0,0,115,6,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,1
ENSG00000130287,7,2,0,6,1,2,4,1,2,24,...,4,2,1,0,0,1,0,1,7,0
ENSG00000113492,0,1,0,5,0,0,1,1,0,1,...,0,2,0,0,0,3,1,1,3,0


In [20]:
ngenes = len(df.index)
nfiles = len(df.columns)
print("genes:%d\trealizations:%d"%(ngenes,nfiles))

genes:223	realizations:407


## files

In [23]:
df_files = pd.DataFrame(index=df.columns)
#df_files.to_csv("files.dat", index=None, header=None)

## Counts to RPK
RPK=$\frac{counts}{lenght}$

In [ ]:
df = pd.concat((df,q_many), ignore_index=False, sort=False, axis=1)
df.head()

In [ ]:
df.drop('type_of_gene',axis=1,inplace=True)

In [ ]:
df = df.div(df['lenght'],axis=0)
df = df.mul(1000,axis=0) #kilobase
df.drop('lenght', axis=1, inplace=True)
df = df.round(2)

In [ ]:
df.head()

In [ ]:
df.to_csv("mainTable_RPK.csv",index=True, header=True)

In [ ]:
lenghts.max()

### null

In [ ]:
df_null=pd.read_csv("nullTable.csv",header=None, index_col=None)
df_null.insert(0,'lenght',q_many['lenght'].values)
df_null.head()

In [ ]:
df_null = df_null.div(df_null['lenght'],axis=0).mul(1000,axis=0) #kilobase
df_null.drop('lenght',axis=1, inplace=True)

In [ ]:
df_null.head()

In [ ]:
df_null.to_csv("nullTable_RPK.csv", header=False, index=False)

## Means sigmas
thresholds are applied for FPKM between 0.1 and 1e5

In [ ]:
%timeit
means = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    means.append(np.average(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
%timeit
variances = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    variances.append(np.var(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
df_mv = pd.DataFrame(data=np.array(means), columns=['mean'], index=df.loc[:,df.keys()[0]].values)
df_mv.insert(1, 'variance', np.array(variances))
df_mv.head()
df_mv.to_csv("meanVariances.csv")

## manipulate meansigmas.csv

In [ ]:
df_mv = pd.read_csv("meanVariances.csv", index_col = [0])
df_mv.head()

### Add type of gene

In [ ]:
gdf = pd.read_csv("genes.txt", index_col=[0])
gdf.head()

In [ ]:
len(gdf)

In [ ]:
df_mv = df_mv.drop(labels='type_of_gene', axis=1)
df_mv.insert(1,'type_of_gene', '')
#for g in df_mv[[df_mv['type_of_gene']==type_of_gene]].index.values
for g in df_mv.index.values:
    df_mv.at[g, 'type_of_gene'] = gdf.loc[g[:15]]['type_of_gene']

In [ ]:
#df_mv = df_mv[df_mv['type_of_gene']==type_of_gene]
df_mv.to_csv("meanVariances.csv", index=True, header=True)

### Add occurence

In [ ]:
odf = pd.read_csv("O.dat", header=None)
#print(len(df_mv[df_mv['type_of_gene']=='protein-coding']))
odf.head()

In [ ]:
len(np.array(odf.loc[:,0].values*5000))

In [ ]:
type_of_gene = 'protein-coding'

In [ ]:
len(gdf[gdf['type_of_gene']==type_of_gene].index)

In [ ]:
len(df_mv.index)

In [ ]:
gdfpc = gdf.loc[gdf['type_of_gene']==type_of_gene]
gdfpc.insert(1, 'occurrence', np.array(odf.loc[:,0].values*5000))
#df_mv.insert(3,'occurrence', 0)
for g in df_mv.index.values:
    df_mv.at[g, 'occurrence'] = gdfpc.loc[g[:15]]['occurrence']

In [ ]:
df_mv.to_csv("meanVariances.csv", header=True, index=True)

## clean floating points

In [ ]:
df.to_csv("mainTable.csv", index=False, header=True, float_format="%.1f")

# File list and properties

In [ ]:
#df_files = pd.read_csv("files.dat", header=[0], index_col=[0])
#df_files.set_index(0,inplace=True)

In [25]:
#df_files = pd.read_csv("files.txt", index_col=[0], header=None)
df_files.insert(0,'primary_site','')
df_files.insert(1,'disease_type', '')
df_files.head()

,primary_site,disease_type
d2587070-cb7d-440d-ae49-52f5077248e6.htseq.counts.gz,,
97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz,,
646d58e7-b7e2-4b2b-bb37-3ca20b601213.htseq.counts.gz,,
0f2b58b4-7d91-48f3-8901-b624e3d6aa51.htseq.counts.gz,,
3eb03db7-efc1-4f20-bbc9-61551905eb9b.htseq.counts.gz,,


In [26]:
df_files.insert(2,'case_id','')
df_files.insert(3,'RPPA Clusters','')

In [27]:
df_colon = pd.read_csv("files_crc.dat", index_col=0)
df_colon.insert(0,'CMS',df_colon.apply(lambda x:np.array(['CMS1','CMS2','CMS3','CMS4'])[x.to_numpy().nonzero()[0]][0] if len(x.to_numpy().nonzero()[0])>0 else 'unknown', axis=1))
df_colon.index=df_colon.index.reindex(np.array([id[:12] for id in df_colon.index.values]))[0]

In [28]:
#df_breast=pd.read_csv("breast.csv")
#df_breast = df_breast.set_index('Complete TCGA ID').loc[:,['PAM50 mRNA','RPPA Clusters']]

In [29]:
#for sample in df_files.index:
for sample in df_files[df_files['RPPA Clusters']==''].index:
    try:
        #sdf=queryFile(sample+'.FPKM.txt.gz')
        sdf=queryFile(sample)
        
        df_files.at[sample,'primary_site'] = sdf['primary_site'][0]
        df_files.at[sample, 'disease_type'] = sdf['disease_type'][0]
        try:
            #df_files.at[sample, 'RPPA Clusters'] = df_breast.loc[sdf['submitter_id'][0],:]['RPPA Clusters']
            df_files.at[sample, 'RPPA Clusters'] = df_colon.loc[sdf['submitter_id'][0],'CMS']
        except:
            df_files.at[sample, 'RPPA Clusters'] = 'Other'
    except:
        for i in range(10):
            try:
                #sdf=queryFile(sample+'%1.0f.FPKM.txt'%i)
                sdf=queryFile(sample)
                df_files.drop(labels=sample, axis=0, inplace=True)
                sample = sample+'%d'%i
                df_files.at[sample,'primary_site'] = sdf['primary_site'][0]
                df_files.at[sample, 'disease_type'] = sdf['disease_type'][0]
                break
            except:
                print("error")

quering...d2587070-cb7d-440d-ae49-52f5077248e6.htseq.counts.gz
quering...97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz
quering...646d58e7-b7e2-4b2b-bb37-3ca20b601213.htseq.counts.gz
quering...0f2b58b4-7d91-48f3-8901-b624e3d6aa51.htseq.counts.gz
quering...3eb03db7-efc1-4f20-bbc9-61551905eb9b.htseq.counts.gz
quering...73ce143e-44f1-49c6-963f-45246d8551de.htseq.counts.gz
quering...09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz
quering...6ae2dd6c-2a39-411f-a1fc-11e0e6e82165.htseq.counts.gz
quering...5f5c42f3-3631-43a3-9774-164eca819389.htseq.counts.gz
quering...b520edf2-42ce-4b64-8575-19e34b30f042.htseq.counts.gz
quering...9daee176-d523-4450-8ac8-0f69d028399e.htseq.counts.gz
quering...59608767-595f-402a-9f7f-356dd6e99a51.htseq.counts.gz
quering...dd9a3ab6-6c93-4b47-9b4e-62e56767054b.htseq.counts.gz
quering...bb8a0f58-11e6-49dd-bdeb-0b7eb2fb05fc.htseq.counts.gz
quering...5ff1be40-73fe-4cd2-9b58-4be53fdf8292.htseq.counts.gz
quering...a42c7fc3-4f41-4b15-b1ec-2108fa916f6a.htseq.co

quering...81903241-4091-45eb-baf1-90b10b727208.htseq.counts.gz
quering...9c64a139-1417-4465-8472-b44182874b0a.htseq.counts.gz
quering...70c7147b-b316-4409-8073-968dfd14abec.htseq.counts.gz
quering...e2916399-9c6c-4d75-a22d-b180ea174644.htseq.counts.gz
quering...a86552ef-96e7-4cb8-9c94-157f57fc6694.htseq.counts.gz
quering...9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
quering...b5bcd3d2-9063-43c1-949e-e3001862855f.htseq.counts.gz
quering...a4de3403-a3ca-4c5c-baf4-99e7068fa0d5.htseq.counts.gz
quering...e86b6f79-cd01-4530-b315-c56e2fb0293f.htseq.counts.gz
quering...59e2ab4e-9529-490d-b12c-9b7f24c8166c.htseq.counts.gz
quering...dd2f8e69-327d-453c-bbea-81b0bea7f977.htseq.counts.gz
quering...a8d7571b-7539-40ac-8be9-209aa54769d2.htseq.counts.gz
quering...ce03135f-f554-4ef7-9af8-c006486516be.htseq.counts.gz
quering...11586a7f-5148-4b6b-b431-686b907f8df9.htseq.counts.gz
quering...7261b656-c79c-4581-a503-15b653e2b5d2.htseq.counts.gz
quering...f0b34dc3-72b8-48d0-a755-354c74405264.htseq.co

quering...20a523cd-58e9-467d-bed5-1bce2a8574f8.htseq.counts.gz
quering...5ccb8845-93d4-4ce2-b97a-e230f1da4611.htseq.counts.gz
quering...c3f9ba64-6946-45d4-984d-b33ef2589603.htseq.counts.gz
quering...c2b0ac40-50ff-47e1-8089-c920a932ed9b.htseq.counts.gz
quering...bd8839a3-4dcb-4688-b051-6f09c18d7fc7.htseq.counts.gz
quering...de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
quering...6fcab48b-754f-4058-9cbf-b6d766eeda35.htseq.counts.gz
quering...bf4c7278-d90c-497e-97af-cc9c68ca1525.htseq.counts.gz
quering...c04b9aba-55bc-4ba9-b761-3fea9f0d4c90.htseq.counts.gz
quering...04404d33-da5c-4477-ae6a-b968332c48c0.htseq.counts.gz
quering...72575c62-b2a8-4fde-8b05-ef6d76df0462.htseq.counts.gz
quering...06c8dfda-b057-47cb-b435-9c2739694f6a.htseq.counts.gz
quering...d6224860-0b09-4d96-8a50-4acb3f72e904.htseq.counts.gz
quering...4e7c6811-88e4-4bb7-a88f-7491dfa6d072.htseq.counts.gz
quering...11320586-6000-4f23-8e68-926175cfadf2.htseq.counts.gz
quering...df959336-a712-47a9-9042-b2b6bc01e813.htseq.co

quering...228845f8-1ad7-4193-a570-e17871798c67.htseq.counts.gz
quering...d265829a-9e49-4e21-ba72-67f5ba406204.htseq.counts.gz
quering...72a79a28-907d-42b8-9722-93448b87c3d2.htseq.counts.gz
quering...e82223ad-33c0-47fd-b0f2-982da64d0cb2.htseq.counts.gz
quering...37e498a7-b147-4c38-a3f5-ed283d57f97a.htseq.counts.gz
quering...c264fe3b-482b-44ec-83a4-73df565663ff.htseq.counts.gz
quering...9e97f017-6855-4d68-bd2a-c4da3ccb1d57.htseq.counts.gz
quering...15864159-be88-41c8-bdef-c2c5927cb1a1.htseq.counts.gz
quering...e2bdca76-7bc5-4143-a6f6-3c6642425c65.htseq.counts.gz
quering...b1868d9d-ef06-4353-883b-230dae23f8b6.htseq.counts.gz
quering...2b5f6764-5a5b-47dc-b0f8-7a548bab2e26.htseq.counts.gz
quering...7324b2d6-dd30-4f99-a101-7c3d497bbcb8.htseq.counts.gz
quering...95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
quering...00cc9b4d-a847-464e-979a-7751e1a87ae3.htseq.counts.gz


In [32]:
df_files.head()

,primary_site,disease_type,case_id,RPPA Clusters
d2587070-cb7d-440d-ae49-52f5077248e6.htseq.counts.gz,Colon,"Cystic, Mucinous and Serous Neoplasms",,Other
97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz,Colon,Adenomas and Adenocarcinomas,,CMS3
646d58e7-b7e2-4b2b-bb37-3ca20b601213.htseq.counts.gz,Colon,Adenomas and Adenocarcinomas,,Other
0f2b58b4-7d91-48f3-8901-b624e3d6aa51.htseq.counts.gz,Colon,Adenomas and Adenocarcinomas,,unknown
3eb03db7-efc1-4f20-bbc9-61551905eb9b.htseq.counts.gz,Colon,Adenomas and Adenocarcinomas,,Other


In [36]:
df_files.to_csv("files.dat", index=True, header=True)

## lenght correlations

In [ ]:
df_mv=pd.read_csv("meanVariances.csv", index_col=[0])

In [ ]:
df_mv.insert(3,'lenght',q_many['lenght'])
df_mv.head()

In [ ]:
df_mv = df_mv.sort_values(by='lenght',axis=0, ascending=False)

# Quantilize

In [ ]:
from tableanalyser import discretize_df_columns

In [ ]:
%timeit
qdf = discretize_df_columns(df)

In [ ]:
qdf.to_csv("mainTable_quartiles.csv", index=True, header=True)

# Filter tissues

In [ ]:
df_files = pd.read_csv("files.dat", index_col=[0])

In [ ]:
np.sort(df_files['primary_site'].unique())

In [ ]:
samples = df_files[df_files['primary_site'].isin(['Blood', 'Brain', 'Colon', 'Uterus, NOS', 'Skin', 'Breast', 'Pancreas', 'Kidney'])].index.values

In [ ]:
df.loc[:,samples].to_csv("main_Table_subtissues.csv",index=True, header=True)